<a href="https://colab.research.google.com/github/Aparimita18/Question_Answering_Model/blob/main/Question_Answering_Model_on_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install PyMuPDF transformers gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 114.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.9 MB/s eta 0:00:00


PDF extraction

In [3]:
import fitz

book = fitz.open("/content/dataset.pdf")

# Extract text from all pages
data = ""
for page in book:
    data += page.get_text()

print(data[:1000])


Speech and Language Processing
An Introduction to Natural Language Processing,
Computational Linguistics, and Speech Recognition
with Language Models
Third Edition draft
Daniel Jurafsky
Stanford University
James H. Martin
University of Colorado at Boulder
Copyright ©2024. All rights reserved.
Draft of January 12, 2025. Comments and typos welcome!
Summary of Contents
I
Fundamental Algorithms for NLP
1
1
Introduction. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
3
2
Regular Expressions, Tokenization, Edit Distance . . . . . . . . . . . . . . .
4
3
N-gram Language Models . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
32
4
Naive Bayes, Text Classiﬁcation, and Sentiment . . . . . . . . . . . . . . . . . 56
5
Logistic Regression . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 77
6
Vector Semantics and Embeddings . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 10

In [4]:
import fitz  # PyMuPDF
import pandas as pd

# Open PDF
doc = fitz.open("dataset.pdf")
page = doc[0]  # First page

# returns list of dicts with positional info
blocks = page.get_text("dict")["blocks"]

# Filter for text blocks
table_data = []
for b in blocks:
    if b["type"] == 0:  # text
        for line in b["lines"]:
            row = []
            for span in line["spans"]:
                row.append(span["text"].strip())
            if row:
                table_data.append(row)

# convert to DataFrame
for row in table_data:
    print(row)

df = pd.DataFrame(table_data)
print(df)


['Speech and Language Processing']
['An Introduction to Natural Language Processing,']
['Computational Linguistics, and Speech Recognition']
['with Language Models']
['Third Edition draft']
['Daniel Jurafsky']
['Stanford University']
['James H. Martin']
['University of Colorado at Boulder']
['Copyright ©2024. All rights reserved.']
['Draft of January 12, 2025. Comments and typos welcome!']
                                                    0
0                      Speech and Language Processing
1     An Introduction to Natural Language Processing,
2   Computational Linguistics, and Speech Recognition
3                                with Language Models
4                                 Third Edition draft
5                                     Daniel Jurafsky
6                                 Stanford University
7                                     James H. Martin
8                   University of Colorado at Boulder
9               Copyright ©2024. All rights reserved.
10  Draft of 

Text Cleaning

In [5]:
!pip install nltk
import re
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [6]:
import re

def clean(text,debug= True):

    text = re.sub(r"-\n", "", text)
    # Replace multiple newlines with a single one
    text = re.sub(r"\n{2,}", "\n\n", text)
    #text = re.sub(r'\[[0-9]+\]', '', text)          # Remove citations like [1]
    text = re.sub(r'\bExercises?\s*\d+(\.\d+)?\b', '', text)

    text = re.sub(r'\b(e\.g\.|i\.e\.)\b', '', text)

    text = re.sub(r'(Fig(?:ure)?\.? ?\d+(\.\d+)?(?:[^\w\n]{0,20})?)', '', text)
    text = re.sub(r"\(?[Tt]he interested reader.*?Section \d+(\.\d+)*.*?\)?", "", text)
    text = re.sub(r"\(?see (Section|Chapter|Subsection) \d+(\.\d+)*.*?\)?", "", text)
    text = re.sub(r"\(?refer to (Section|Chapter|Subsection) \d+(\.\d+)*.*?\)?", "", text)
    text = re.sub(r"\(?(as|as we) (see|saw) (in )?(Section|Chapter|Subsection) \d+(\.\d+)*.*?\)?", "", text)
    text = re.sub(r'^\s*[\*\-•]+.*$', '', text, flags=re.MULTILINE)
    #paragraphs = re.split(r'\n?(?=\d+(\.\d+)*\s+[^\n]+)', text)
    text = re.sub(r'^\s*\d+(\.\d+)*\s+[•\-–—]\s+.+?\d\s*$', '', text, flags=re.MULTILINE)




    lines = text.split('\n')
    cleaned_lines = []
    for line in lines:
        line = line.strip()

        # Regular expressions to handle various cases
        if re.match(r'^\d+\s+Chapter\s+\d+', line, re.IGNORECASE):
            continue
        if re.match(r'^Chapter\s+\d+', line, re.IGNORECASE):
            continue
        if re.fullmatch(r'\d+\s*[•\-–—]?', line) or re.fullmatch(r'[•\-–—]?\s*\d+', line):
            continue
        if re.fullmatch(r'\d+', line):
            continue
        if re.fullmatch(r'\d*\.\d+', line):
            continue
        if re.fullmatch(r'(Figure|Table|Section|Subsection)\s+\d+(\.\d+)*', line, re.IGNORECASE):
            continue
        if re.fullmatch(r'^•\s.*\s[\d.]+$',line):
            continue
        if re.search(r"(Speech and Language Processing|Jurafsky|Martin|Draft of \d{4})", line, re.IGNORECASE):
            continue
        if re.match(r'^[>\-•*]?\s*[A-Z\s]+$', line):
            continue
        if re.fullmatch(r'[\d\s•\-–—]*', line):
            continue
        if re.search(r"(C HAPTER|Speech and Language Processing|Jurafsky|Martin|Draft of \d{4}|CHAPTER \d+|N- GRAM|SPEECH RECOGNITION)", line, re.IGNORECASE):
            continue
        if re.fullmatch(r'[•\-–—]\s+[A-Z\s]{3,}', line) or line.isupper():
            continue
        if re.search(r'(Figure|Fig\.|Table|Box|Image|Diagram)\s*\d+(\.\d+)?', line, re.IGNORECASE):
            continue
        if re.fullmatch(r'[\[\(]?(image|diagram|fig|figure|illustration)[\]\)]?', line, re.IGNORECASE):
            continue
        if len(re.findall(r'[A-Za-z]', line)) < 4:
            continue
        if re.fullmatch(r'[•\-–—]\s+[A-Z\s]', line):
            continue

        cleaned_lines.append(line)

    cleaned_text = "\n".join(cleaned_lines)

    return cleaned_text


In [7]:
def splitpara(text):
    paragraphs = []
    current = ""

# Define known footer patterns (can extend this as needed)
    footer_patterns = [
        r'^For example if we use both begin.*',
        r'^Speech and Language Processing.*',
        r'^Jurafsky.*',
        r'^Draft of \d{4}.*',
        r'^Page \d+.*',
        r'^Figure \d+.*',
        r'^.*\b\d{3,}\b.*$'  # Lines with page numbers
    ]

    # Clean up footers
    lines = text.split('\n')
    cleaned_lines = []
    for line in lines:
        if any(re.match(pat, line.strip()) for pat in footer_patterns):
            continue
        cleaned_lines.append(line)

    # Join back into cleaned text
    cleaned_text = '\n'.join(cleaned_lines)

    # paragraph splitting based on subsection heading and punctuation
    lines = cleaned_text.split('\n')
    for line in lines:
        line = line.strip()
        if not line:
            continue

        # Remove figure/image/table mentions
        line = re.sub(r'(Fig(?:ure)?\.?\s*\d+(\.\d+)?(?:[^\w\n]{0,20})?)', '', line)
        line = re.sub(r'(Table\s*\d+(\.\d+)?(?:[^\w\n]{0,20})?)', '', line)
        line = re.sub(r'(Box\s*\d+(\.\d+)?(?:[^\w\n]{0,20})?)', '', line)
        line = re.sub(r'\(?(see|refer to)?\s*(Fig(?:ure)?|Table|Box)\s*\d+(\.\d+)?\)?', '', line, flags=re.IGNORECASE)


        if re.match(r'^\d+(\.\d+)+\s+[A-Za-z]', line):
            if current.strip():
                paragraphs.append(current.strip())
                current = ""
            paragraphs.append(line.strip())
            continue

        # If line ends with a period

        if line.endswith('.'):
            current += " " + line
            paragraphs.append(current.strip())
            current = ""
        else:
            current += " " + line

        # Add remaining paragraph
        if current.strip():
            paragraphs.append(current.strip())

    final_paragraphs = []
    for para in paragraphs:
        sentences = sent_tokenize(para)
        merged = ''.join([sent.strip() for sent in sentences if sent.strip()])
        final_paragraphs.append(merged)

    return final_paragraphs

    '''#splitting on whitespace i.e 2 or more newlines
    final_paragraphs = []
    for para in paragraphs:
        chunks = re.split(r'\n\s*\n', para)
        final_paragraphs.extend([chunk.strip() for chunk in chunks if chunk.strip()])

    return final_paragraphs
    #return paragraphs'''

In [8]:
cleaned_text = clean(data)
paragraphs = splitpara(cleaned_text)

import textwrap
print(f"Total paragraphs: {len(paragraphs)}")
print(textwrap.fill(paragraphs[0]))

Total paragraphs: 25123
An Introduction to Natural Language Processing,


In [9]:
import pandas as pd

df = pd.DataFrame(paragraphs, columns=["text"])
df.to_csv("dataset.csv", index=False,sep="|", encoding="utf-8", header=True)
print("Saved as dataset.csv")

from google.colab import files
#files.download("dataset.csv")

Saved as dataset.csv


In [10]:
print(textwrap.fill(df['text'][1839]))

These unseen sequences or zeros—sequences that don’t occur in the
training set


## **Re-check if the data extraction & cleaning is done properly**

In [11]:

#list of all paragraphs
paragraphs = df['text']
print(paragraphs)
print("\n")
print(len(paragraphs))

0          An Introduction to Natural Language Processing,
1        An Introduction to Natural Language Processing...
2        An Introduction to Natural Language Processing...
3        An Introduction to Natural Language Processing...
4        An Introduction to Natural Language Processing...
                               ...                        
25118    fragment of word, 13 Gaussian prior on weights...
25119    fragment of word, 13 Gaussian prior on weights...
25120    fragment of word, 13 Gaussian prior on weights...
25121    fragment of word, 13 Gaussian prior on weights...
25122    fragment of word, 13 Gaussian prior on weights...
Name: text, Length: 25123, dtype: object


25123


In [12]:
print(f"Total pages in PDF: {len(book)}")

Total pages in PDF: 599


In [13]:
print(f"Total paragraphs extracted: {len(paragraphs)}")

Total paragraphs extracted: 25123


In [14]:
print(textwrap.fill(df['text'][1253]))

When entering a value in each cell, we mark which of the three
neighboring cells we came from with up to three arrows.After the table
is full we compute an alignment (minimum edit path) by using a
backtrace, starting at the 8 in the lower-right corner and following
the arrows back.The sequence of bold cells represents one possible
minimum cost alignment between the two strings, again using
Levenshtein distance with cost of 1 for While we worked our example
with simple Levenshtein distance, the algorithm in allows arbitrary
weights on the operations.For spelling correction, for example,
substitutions are more likely to happen between letters that are next
to each other on the keyboard.The Viterbi algorithm is a probabilistic
extension of minimum edit distance.Instead of computing the “minimum
edit distance” between


In [15]:
import random

for _ in range(5):
    parano = random.randint(0, len(paragraphs)-1)
    print(f"--- Paragraph {parano} ---")
    print(textwrap.fill(paragraphs[parano]))
    print("\n\n")

--- Paragraph 18925 ---
Computational linguistics/NLP thus differs in its use of the term
reference from the ﬁeld of formal



--- Paragraph 14641 ---
Treebanks are generally made by running a parser over each sentence
and then having the resulting parse hand-corrected by human
linguists.shows sentences from the Penn Treebank project, which
includes various treebanks in Penn Treebank English, Arabic, and
Chinese.The Penn Treebank part-of-speech tagset was deﬁned of LISP-
style parenthesized notation for trees is extremely common and
resembles the bracketed notation we saw earlier in (18.1).For those
who are not familiar with it we show a standard node-and-line tree
representation in (NP-SBJ (DT That) (JJ cold) (, ,) (JJ empty) (NN
sky) ) (VP (VBD was) (ADJP-PRD (JJ full) (PP (IN of) (NP (NN fire) (CC
and)



--- Paragraph 21926 ---
Glaese, A., N. McAleese, M. Trebacz, J. Aslanides, V. Firoiu, T.
Ewalds, M. Rauh, L. Weidinger, M. Chadwick, P. Thacker, L.
CampbellGillingham, J. Uesato, P

In [16]:

chapters = [
  "Introduction",
  "Regular Expressions, Tokenization, Edit Distance",
  "N-gram Language Models",
  "Naive Bayes, Text Classification, and Sentiment",
  "Logistic Regression",
  "Vector Semantics and Embeddings",
  "Neural Networks",
  "RNNs and LSTMs",
  "The Transformer",
  "Large Language Models",
  "Masked Language Models",
  "Model Alignment, Prompting, and In-Context Learning",
  "Machine Translation",
  "Question Answering, Information Retrieval, and RAG",
  "Chatbots & Dialogue Systems",
  "Automatic Speech Recognition and Text-to-Speech",
  "Annotating Linguistic Structure",
  "Sequence Labeling for Parts of Speech and Named Entities",
  "Context-Free Grammars and Constituency Parsing",
  "Dependency Parsing",
  "Information Extraction: Relations, Events, and Time",
  "Semantic Role Labeling",
  "Lexicons for Sentiment, Affect, and Connotation",
  "Coreference Resolution and Entity Linking",
  "Discourse Coherence",
]

In [17]:
!pip install fuzzywuzzy

In [18]:

from fuzzywuzzy import fuzz

/usr/local/lib/python3.11/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [19]:

matches = {}
for section in chapters :
    found = any(fuzz.partial_ratio(section.lower(), para.lower()) > 25 for para in paragraphs)

    #found = any(section.lower() in para.lower() for para in paragraphs)
    matches[section] = "✅ Found" if found else "❌ Not Found"

for k, v in matches.items():
    print(f"{k}: {v}")

Introduction: ✅ Found
Regular Expressions, Tokenization, Edit Distance: ✅ Found
N-gram Language Models: ✅ Found
Naive Bayes, Text Classification, and Sentiment: ✅ Found
Logistic Regression: ✅ Found
Vector Semantics and Embeddings: ✅ Found
Neural Networks: ✅ Found
RNNs and LSTMs: ✅ Found
The Transformer: ✅ Found
Large Language Models: ✅ Found
Masked Language Models: ✅ Found
Model Alignment, Prompting, and In-Context Learning: ✅ Found
Machine Translation: ✅ Found
Question Answering, Information Retrieval, and RAG: ✅ Found
Chatbots & Dialogue Systems: ✅ Found
Automatic Speech Recognition and Text-to-Speech: ✅ Found
Annotating Linguistic Structure: ✅ Found
Sequence Labeling for Parts of Speech and Named Entities: ✅ Found
Context-Free Grammars and Constituency Parsing: ✅ Found
Dependency Parsing: ✅ Found
Information Extraction: Relations, Events, and Time: ✅ Found
Semantic Role Labeling: ✅ Found
Lexicons for Sentiment, Affect, and Connotation: ✅ Found
Coreference Resolution and Entity Linki

##**Chunk text into tokens for BERT**

In [20]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [21]:

def chunk_paragraphs(paragraphs, tokenizer, max_length=512, stride=64):
    """
    Tokenizes and chunks paragraphs into BERT-compatible segments using a sliding window.
    Returns a list of dictionaries with input_ids, attention_mask, and original text.
    """
    chunked = []

    for idx, para in enumerate(paragraphs):
        encodings = tokenizer(
            para,
            return_overflowing_tokens=True,
            truncation=True,
            max_length=max_length,
            stride=stride,
            return_attention_mask=True,
            #return_offsets_mapping=True,
            padding=False
        )

        for i in range(len(encodings["input_ids"])):
            chunk_text = tokenizer.decode(encodings["input_ids"][i], skip_special_tokens=True)
            chunked.append({
                "para_index": idx,
                "chunk_index": i,
                "input_ids": encodings["input_ids"][i],
                "attention_mask": encodings["attention_mask"][i],
                "text": chunk_text
            })

    return chunked

def chunksfile(chunks, output_path):
    df = pd.DataFrame(chunks)
    df.to_csv(output_path, index=False)

# Pipeline Execution
pdf_path = "dataset.pdf"
outputfile = "chunked_data.csv"

# 1. Extract and preprocess
book = fitz.open(pdf_path)
data = ""
for page in book:
    data += page.get_text()
cleaned_text = clean(data)
paragraphs = splitpara(cleaned_text)

# 2. Chunk using BERT tokenizer
chunked_data = chunk_paragraphs(paragraphs, tokenizer)

# 3. Save to CSV
chunksfile(chunked_data, outputfile)

print(f"Chunked data saved to {outputfile}")


Chunked data saved to chunked_data.csv


In [22]:
# Use your previously prepared paragraphs
paragraphs = df['text'].tolist()  # assuming df from earlier

# Example output
print(f"Total chunks: {len(chunked_data)}")
print("Sample chunk text:\n", chunked_data[1000]["text"])


Total chunks: 25932
Sample chunk text:
 ( thus the frequencies in the test data don ’ t play a role, just the frequencies in the training data ). so ﬁrst we segment each test sentence word into characters. then we apply the ﬁrst rule : replace every instance of e r in the test corpus with er, and then the second rule : replace every instance of er in the test corpus with er, and so on. by the end, if the test corpus contained the character sequence n e w e, it would be tokenized as a full word. but the characters of a new ( unknown ) word like l o w e r would be merged into the two tokens low er.


Generate manual QA pairs

In [23]:

import pandas as pd

# Load paragraph dataset
paragraphs = pd.read_csv('/content/dataset.csv', sep = '|')
paragraphs = paragraphs.dropna().reset_index(drop=True)
#paragraphs.head()


In [24]:


'''# Create an empty DataFrame for manual QA
manual_qa = pd.DataFrame(columns=['question', 'answer'])
#example
new_row = pd.DataFrame([{'question': 'What is instruction tuning?',
                               'answer': 'Instruction tuning (short for instruction finetuning, and sometimes even shortened to instruct tuning) is a method for making an LLM better at following instructions. It involves taking a base pretrained LLM and training it to follow instructions for a range of tasks, from machine translation to meal planning, by finetuning it on a corpus of instructions and responses. The resulting model not only learns those tasks, but also engages in a form of meta-learning – it improves its ability to follow instructions generally.Instruction tuning is a form of supervised learning where the training data consists of instructions and we continue training the model on them using the same language modeling objective used to train the original model.'
                               }])

manual_qa = pd.concat([manual_qa, new_row], ignore_index=True)

# Save once done
#manual_qa.to_csv('manual.csv', index=False)
'''

"# Create an empty DataFrame for manual QA\nmanual_qa = pd.DataFrame(columns=['question', 'answer'])\n#example\nnew_row = pd.DataFrame([{'question': 'What is instruction tuning?',\n                               'answer': 'Instruction tuning (short for instruction finetuning, and sometimes even shortened to instruct tuning) is a method for making an LLM better at following instructions. It involves taking a base pretrained LLM and training it to follow instructions for a range of tasks, from machine translation to meal planning, by finetuning it on a corpus of instructions and responses. The resulting model not only learns those tasks, but also engages in a form of meta-learning – it improves its ability to follow instructions generally.Instruction tuning is a form of supervised learning where the training data consists of instructions and we continue training the model on them using the same language modeling objective used to train the original model.'\n                              

In [25]:
import pandas as pd
data = pd.read_csv('/content/manual.csv',sep = ',')
data

,question,answer
0,What is instruction tuning?,Instruction tuning (short for instruction fine...
1,Why did the parallelogram method receive more ...,parallelogram method received more modern atte...
2,Why is training more challenging in the mentio...,Training is trickier in the mention-ranking mo...
3,What is lemmatization in text normalization?,Lemmatization is the task of determining that ...
4,Why is lemmatization important for processing ...,Lemmatization is essential for processing morp...
...,...,...
154,What is the retriever/reader architecture in q...,"In the retriever/reader architecture, the retr..."
155,What is retrieval-augmented generation in the ...,Retrieval-augmented generation involves using ...
156,How can QA systems be evaluated when only a si...,QA systems can be evaluated by exact match wit...
157,What evaluation metric is used for free text a...,"For free text answers, QA can be evaluated usi..."


Generate automatic QA pairs

In [26]:

!pip install sentencepiece

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import pandas as pd
from tqdm import tqdm
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [27]:
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [28]:
!pip install transformers
!pip install sentencepiece


In [ ]:
!pip install --upgrade  transformers sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.4/481.4 kB 18.1 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.28.1
    Uninstalling huggingface-hub-0.28.1:
      Successfully uninstalled huggingface-hub-0.28.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.3
    Uninstalling transformers-4.48.3:
      Successfully uninstalled transformers-4.48.3


In [ ]:
!pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install cloud-tpu-client
!pip install --upgrade torch_xla

Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 1.4 MB/s eta 0:00:00
^C
^C
^C


In [29]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import random

# Load tokenizer & model
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [30]:
def generate_question_t5(context):
    input_text = "generate question: " + context.strip()
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)

    output_ids = model.generate(input_ids,
                                max_length=64,
                                num_beams=4,
                                early_stopping=True)

    question = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return question


In [31]:

import pandas as pd

# Load your dataset
df = pd.read_csv("dataset.csv",sep='|')

def chunk_paragraphs(paragraphs, tokenizer, max_length=512, stride=64):
    chunks = []
    for para in paragraphs:
        # Tokenize the paragraph and split into chunks
        encoding = tokenizer(para, return_tensors="pt", max_length=max_length, stride=stride, truncation=True)
        for input_ids in encoding['input_ids']:
            chunks.append([input_ids.tolist()])  # Wrap in a list to represent a row
    return chunks

        #chunks.extend(encoding['input_ids'])
    #return chunks'''

chunked = chunk_paragraphs(df['text'].tolist(), tokenizer)
chunked_df = pd.DataFrame(chunked,columns = ['text'])
chunked_df.to_csv("chunked_data.csv", sep='|', index=False, encoding='utf-8')


/usr/local/lib/python3.11/dist-packages/transformers/models/t5/tokenization_t5.py:289: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


In [32]:
from transformers import pipeline

# Load the QA pipeline using a pre-trained BERT model
qa_pipeline = pipeline("question-answering", model="deepset/bert-base-cased-squad2")

def generate_answer_bert(context, question):

    result = qa_pipeline(question=question, context=context)
    return result['answer']


config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at deepset/bert-base-cased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/152 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
import json
import pandas as pd
from tqdm import tqdm

sampled_df = chunked_df.sample(n=1000, random_state=42).reset_index(drop=True)
sampled_df['text'] = sampled_df['text'].astype(str)

num_batches = 5
batch_size = 200

for i in range(num_batches):
    qa_pairs = []
    empty_q_count = 0
    batch_df = sampled_df[i * batch_size: (i + 1) * batch_size]

    for para in tqdm(batch_df['text'], desc=f"Batch {i}"):
        try:

            if not isinstance(para,str):
              para = str(para)
            question = generate_question_t5(para)
            if not question.strip():
                empty_q_count += 1
                continue  # Skip this if question is empty

            answer = generate_answer_bert(para, question)
            qa_pairs.append({
                "context": para,
                "question": question,
                "answer": answer
            })

        except Exception as e:
            print(f"Error in batch {i}: {e}")

    print(f"Empty questions skipped in batch {i}: {empty_q_count}")

    # Save the QA pairs for this batch
    with open(f"automatic{i}.json", "w", encoding="utf-8") as f:
        json.dump(qa_pairs, f, ensure_ascii=False, indent=2)

    pd.DataFrame(qa_pairs).to_csv(f"automatic{i}.csv", sep="|", index=False, encoding="utf-8")


Batch 0:   2%|▏         | 3/200 [00:18<19:55,  6.07s/it]


KeyboardInterrupt: 

In [ ]:
import json
import pandas as pd
from tqdm import tqdm

# Sample and ensure text is string
sampled_df = chunked_df.sample(n=500, random_state=42).reset_index(drop=True)
sampled_df['text'] = sampled_df['text'].astype(str)

num_batches = 5
batch_size = 100

for i in range(num_batches):
    qa_pairs = []
    empty_q_count = 0
    batch_df = sampled_df[i * batch_size: (i + 1) * batch_size]

    for para in tqdm(batch_df['text'], desc=f"Batch {i}"):
        try:
            para = str(para).strip()
            question = generate_question_t5(para)

            if isinstance(question, list):
                question = " ".join(map(str, question))
            question = str(question).strip()

            if not question:
                empty_q_count += 1
                continue

            answer = generate_answer_bert(para, question)
            if isinstance(answer, dict) and 'answer' in answer:
                answer = answer['answer']
            if isinstance(answer, list):
                answer = " ".join(map(str, answer))
            answer = str(answer).strip()
            qa_pairs.append({
                "context": str(para),
                "question": str(question),
                "answer": str(answer)
            })

        except Exception as e:
            print(f"Error in batch {i}: {e}")

    print(f"Empty questions skipped in batch {i}: {empty_q_count}")

    # Save as JSON
    with open(f"automatic{i}.json", "w", encoding="utf-8") as f:
        json.dump(qa_pairs, f, ensure_ascii=False, indent=2)

    # Save as CSV after ensuring all fields are strings
    df = pd.DataFrame(qa_pairs)
    df['context'] = df['context'].astype(str)
    df['question'] = df['question'].astype(str)
    df['answer'] = df['answer'].astype(str)
    df.to_csv(f"automatic{i}.csv", sep="|", index=False, encoding="utf-8")


Batch 0: 100%|██████████| 100/100 [06:48<00:00,  4.08s/it]


Empty questions skipped in batch 0: 0


Batch 1: 100%|██████████| 100/100 [06:27<00:00,  3.87s/it]


Empty questions skipped in batch 1: 0


Batch 2: 100%|██████████| 100/100 [06:47<00:00,  4.08s/it]


Empty questions skipped in batch 2: 0


Batch 3: 100%|██████████| 100/100 [06:28<00:00,  3.89s/it]


Empty questions skipped in batch 3: 0


Batch 4:   2%|▏         | 2/100 [00:08<06:30,  3.99s/it]

In [34]:


import pandas as pd

manual_df = pd.read_csv("manual.csv", sep=",")

#Load automatic QA files
automatic_dfs = []
for i in range(5):
    automatic_df = pd.read_csv(f"automatic{i}.csv", sep="|")
    automatic_dfs.append(automatic_df)

#Concatenate automatic QA files into one
automatic_combined_df = pd.concat(automatic_dfs, ignore_index=True)

#Combine manual and automatic QA pairs into one
combined_df = pd.concat([manual_df, automatic_combined_df], ignore_index=True)

#Shuffle the combined dataset
combined_df = combined_df.sample(frac=1, random_state=42).reset_index(drop=True)

#Save the combined dataset to a CSV
combined_df.to_csv("combined_qa.csv", sep="|", index=False, encoding="utf-8")

# 7. Print message confirming that the combined file is saved
print("Combined QA pairs saved to 'combined_qa.csv'")


Combined QA pairs saved to 'combined_qa.csv'


In [36]:
import pandas as pd
df = pd.read_csv('/content/combined_qa.csv',sep = '|')
df

,question,answer,context
0,"57, 57, 3374, 53, 3, 9, 17222, 3317, 5259, 44,...",1,"[825, 57, 3374, 53, 3, 9, 17222, 3317, 5259, 4..."
1,"6803, 10, 1300, 634, 982, 284, 43, 192, 2251, ...","813, 15358","[11860, 10, 363, 47, 396, 422, 58, 188, 29, 7,..."
2,What is the main feature of transformers in co...,Transformers are non-recurrent networks that r...,NaN
3,not_duplicate,"6, 3, 99, 1","[86, 14156, 5, 6982, 34, 19, 8, 3832, 41, 23, ..."
4,"9886, 97, 7910, 5, 1649, 9886, 97, 7910, 5, 16...","16482, 24, 3, 122, 19, 3","[852, 62, 22, 60, 1065, 12, 217, 8, 423, 13850..."
...,...,...,...
1153,"86, 5354, 1343, 1249, 4057, 6493, 13774, 6, 21...","3, 9, 1","[86, 5354, 1343, 1249, 4057, 6493, 13774, 6, 2..."
1154,"26093, 6, 2841, 6, 7163, 61, 11, 1","61, 11, 1","[242, 6493, 1693, 62, 2389, 43, 220, 2287, 41,..."
1155,"13, 1448, 6, 1179, 12520, 7, 10488, 1884, 30, ...","1075, 6, 3381","[12071, 13, 1448, 6, 1179, 12520, 7, 10488, 18..."
1156,not_duplicate,1,"[822, 366, 410, 493, 63, 14549, 456, 2852, 101..."
